# Modelo de LSTM para previsão do ibovespa

## Bibliotecas

In [31]:
# Usando a versão 3.10.11 do python

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from pathlib import Path
from datetime import timedelta
import os
pd.options.mode.chained_assignment = None

## Ler base

In [32]:
# Caminho da base
base_diretório = Path(os.getcwd())
diretório_princapl = base_diretório.parent
caminho = diretório_princapl / 'Base' / 'Base Ibovespa Tratada.csv'

# Carrega a base
acao = pd.read_csv(caminho)
acao['Data'] = pd.to_datetime(acao['Data'])

# Ordena por data
acao = acao.sort_values(by="Data")

acao

,Data,Fechamento,Abertura,Máxima,Mínima,Variacao (%)
0,2015-01-05,47517.0,48512.0,48512.0,47264.0,-0.0205
1,2015-01-06,48001.0,47517.0,48061.0,47338.0,0.0102
2,2015-01-07,49463.0,48006.0,49882.0,48006.0,0.0305
3,2015-01-08,49943.0,49463.0,50261.0,49017.0,0.0097
4,2015-01-09,48840.0,49955.0,49955.0,48501.0,-0.0221
...,...,...,...,...,...,...
2609,2025-07-14,135299.0,136187.0,136187.0,134840.0,-0.0065
2610,2025-07-15,135250.0,135298.0,136022.0,134380.0,-0.0004
2611,2025-07-16,135511.0,135250.0,135641.0,134265.0,0.0019
2612,2025-07-17,135565.0,135515.0,135792.0,135016.0,0.0004


In [33]:
# # Criando a figura
# fig = go.Figure()

# # Adicionando a linha do fechamento
# fig.add_trace(go.Scatter(
#     x=acao['Data'],
#     y=acao['Fechamento'],
#     mode='lines',
#     name='Fechamento',
#     line=dict(color='blue')
# ))

# # Layout com interações
# fig.update_layout(
#     title=dict(
#         text='Série Histórica Ibovespa',
#         x=0.5,
#         xanchor='center'
#     ),
#     xaxis_title='Data',
#     yaxis_title='Valor Fechamento',
#     xaxis=dict(
#         rangeselector=dict(
#             buttons=list([
#                 dict(count=1, label='1m', step='month', stepmode='backward'),
#                 dict(count=3, label='3m', step='month', stepmode='backward'),
#                 dict(count=6, label='6m', step='month', stepmode='backward'),
#                 dict(count=1, label='1a', step='year', stepmode='backward'),
#                 dict(step='all', label='Tudo')
#             ])
#         ),
#         rangeslider=dict(visible=True),
#         type='date'
#     )
# )

# # Mostrar gráfico da série histórica do IBOVESPA
# fig.show()

## Aplicando modelo de ML

In [34]:
# Define data de ontem e anteontem
ultimo_dia = acao['Data'].max()
penultimo_dia = acao['Data'].iloc[-2]

print(f'Ontem: {ultimo_dia}\nAnteontem: {penultimo_dia}')

Ontem: 2025-07-18 00:00:00
Anteontem: 2025-07-17 00:00:00


In [35]:
# Filtrando base que vai ser usada para treino até o penultimo dia da base
acao_treino = acao#[acao['Data'] < ultimo_dia]
cotacao_treino = acao_treino['Fechamento'].to_numpy().reshape(-1, 1)

In [36]:
# Armazenar tamanho dos dados de treinamento
tamanho_dados_treinamento = int(len(cotacao_treino) * 1)
tamanho_dados_treinamento

2614

In [37]:
# Escala os dados
escalador = MinMaxScaler(feature_range=(0, 1))
dados_escalados = escalador.fit_transform(cotacao_treino)

In [38]:
# Cria dados de entrada para o modelo
treinamento_x = []
treinamento_y = []

for i in range(60, len(dados_escalados)):
    treinamento_x.append(dados_escalados[i-60:i, 0])
    treinamento_y.append(dados_escalados[i, 0])

In [39]:
treinamento_x, treinamento_y = np.array(treinamento_x), np.array(treinamento_y)
treinamento_x = treinamento_x.reshape(treinamento_x.shape[0], treinamento_x.shape[1], 1)

In [40]:
# Define e treina o modelo
modelo = Sequential()
modelo.add(LSTM(100, return_sequences=True, input_shape=(treinamento_x.shape[1], 1)))
modelo.add(Dropout(0.2))
modelo.add(LSTM(50, return_sequences=False))
modelo.add(Dropout(0.2))
modelo.add(Dense(25))
modelo.add(Dense(1))
modelo.compile(optimizer="adam", loss="mean_squared_error")
modelo.fit(treinamento_x, treinamento_y, batch_size=10, epochs=20)

Epoch 1/20


c:\Users\Arq. Andrea\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



256/256 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.0325
Epoch 2/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0029
Epoch 3/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.0021
Epoch 4/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.0018
Epoch 5/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 0.0015
Epoch 6/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 0.0012
Epoch 7/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 0.0011
Epoch 8/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 0.0010
Epoch 9/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 0.0010
Epoch 10/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 9.1087e-04
Epoch 11/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0010
Epoch 12/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 9.2036e-04
Epoch 13/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 9.7529e-04
Epoch 14/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.0011
Epoch 15/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4

In [41]:
# Pega os últimos 60 dias até ANTEONTEM para previsão
ultimos_60 = dados_escalados[-60:]
entrada_predicao = ultimos_60.reshape(1, 60, 1)

In [42]:
predicao = modelo.predict(entrada_predicao)
predicao = escalador.inverse_transform(predicao)

print(f'Valor previsto: {predicao[0][0]:.2f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
Valor previsto: 132622.47


# Métricas de avaliação do nosso modelo de ML

### Acurácia direcional

In [43]:


# ---- Descomentar as quatro linhas de baixo caso for fazer teste do valor previsto do ultimo dia da base ----
valor_real = acao[acao['Data'] == ultimo_dia]['Fechamento'].values[0]
valor_anterior = acao[acao['Data'] == penultimo_dia]['Fechamento'].values[0]
variacao_real = (valor_real - valor_anterior) / valor_anterior * 100
variacao_predita = (predicao[0][0] - valor_anterior) / valor_anterior * 100
# -----------------------------------------------------------------------------

# ---- Descomentar as quatro linhas de baixo caso for prever do dia seguinte ----
#valor_real = 137002	# Valor de fechamento do ibovespa no dia 04/06/2025 (dia seguinte do ultimo que tem na base)
#valor_anterior = acao[acao['Data'] == ultimo_dia]['Fechamento'].values[0]
#variacao_real = (valor_real - valor_anterior) / valor_anterior * 100
#variacao_predita = (predicao[0][0] - valor_anterior) / valor_anterior * 100
# -----------------------------------------------------------------------------

# Direção correta?
direcao_correta = (variacao_real * variacao_predita) > 0  # ambos com mesmo sinal

# Criar df final
df_previsao = pd.DataFrame([{
    'Data': ultimo_dia.date(), # Data do penultimo dia da base
    #'Data Previsão': ultimo_dia.date() + timedelta(days=1), # Data do dia seguinte do ultimo dia da base
    'Valor Anterior': f"{valor_anterior:.2f}",
    'Valor Real': f"{valor_real:.2f}",
    'Valor Previsto': f"{predicao[0][0]:.2f}",
    'Variação Real (%)': f"{variacao_real:.3f}",
    'Variação Prevista (%)': f"{variacao_predita:.3f}",
    'Direção Correta?': 'Sim' if direcao_correta else 'Não'
}])

# Ajustar colunas numéricas
colunas_para_converter = ['Valor Real', 'Valor Previsto', 'Variação Real (%)', 'Variação Prevista (%)']
for coluna in colunas_para_converter:
    df_previsao[coluna] = df_previsao[coluna].astype(float)
    
df_previsao

,Data,Valor Anterior,Valor Real,Valor Previsto,Variação Real (%),Variação Prevista (%),Direção Correta?
0,2025-07-18,135565.00,133606.0,132622.47,-1.445,-2.171,Sim


### Mean Absolute Error (MAE) e Mean Absolute Percentage Error (MAPE)

In [44]:
# A média das diferenças absolutas entre as previsões e os valores reais. Penaliza todos os erros da mesma forma
mae = mean_absolute_error(df_previsao['Valor Real'], df_previsao['Valor Previsto'])
print(f"Mean Absolute Error (MAE): {mae:.2f}")

# Calcular o MAPE
mape = mean_absolute_percentage_error(df_previsao['Valor Real'], df_previsao['Valor Previsto'])
print(f"Mean Absolute Percentage Error (MAPE): {mape:.6f} = {100*mape:.4f}%")

Mean Absolute Error (MAE): 983.53
Mean Absolute Percentage Error (MAPE): 0.007361 = 0.7361%


### Root Mean Squared Error (RMSE)

In [ ]:
# A raiz quadrada da média dos erros ao quadrado. Dá mais peso a grandes erros
rmse = np.sqrt(mean_squared_error(df_previsao['Valor Real'], df_previsao['Valor Previsto']))
mean_price = df_previsao['Valor Real'].mean()

print(f"RMSE: {rmse:.2f}")
print(f"RMSE percentual: {(rmse / mean_price):.6f} = {100*(rmse / mean_price):.4f}%")

RMSE: 983.53
RMSE percentual: 0.007361 = 0.7361%
